In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:

if not os.path.exists('glove.6B.100d.txt'):
    !wget http://nlp.stanford.edu/data/glove.6B.zip
    !unzip -o glove.6B.zip
    !rm glove.6B.zip

--2025-07-11 08:37:34--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2025-07-11 08:37:34--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2025-07-11 08:37:34--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glov

In [3]:
# ONLY RUN ONCE
!pip install gensim spacy scikit-learn
!python -m spacy download en_core_web_md

import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models import KeyedVectors
import spacy
import requests
from io import StringIO
import os


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 1.6 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of thinc to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 6.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 38.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.7/11.7 MB 42.7 MB/s eta 0:00:0000:010:01
  Attempting uninstall: blis
    Found existing installation: blis 1.3.0
    Uninstalling blis-1.3.0:
      Successfully uninstalled blis-1.3.0
  Attempting uninstall: thinc
    Found existing installation: thinc 8.3.6
    Uninstalling thinc-8.3.6:
      Successfully uninstalled thinc-8.3.6
  Attempting uninstall: scipy
    Found existing installation: scipy 1.15.3
    Uninstalling scipy-1.15.3:
      Successfully uninstalled scipy-1.15.3
ERROR: pip's dependency resolver does not currently take into account a

In [4]:
# i hate chess
texts = [
    "I hate chess, i know what chess is all about, it is about memorization and knowing patterns in positions",
    "never play 1. d4 d5 2. Bf4 also known as the stupid london system, you kill the beauty of the game you moron"
]

In [10]:
print("am i loading?")
word_vectors = KeyedVectors.load_word2vec_format('glove.6B.300d.txt', binary=False, no_header=True)
print(f"Loaded {len(word_vectors)} word vectors")

am i loading?
Loaded 400000 word vectors


In [6]:
# Average Embedding function
def average_embedding(text):
    words = text.lower().split()
    vectors = [word_vectors[word] for word in words if word in word_vectors]
    return np.mean(vectors, axis=0) if vectors else np.zeros(word_vectors.vector_size)

In [7]:
# TF-IDF Weighted Embedding
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(texts)
vocab = vectorizer.get_feature_names_out()

def tfidf_weighted_embedding(text):
    words = text.lower().split()
    text_tfidf = vectorizer.transform([text])
    tfidf_scores = dict(zip(vocab, text_tfidf.toarray()[0]))
    
    weighted_vectors = []
    for word in words:
        if word in word_vectors and word in tfidf_scores:
            weighted_vectors.append(tfidf_scores[word] * word_vectors[word])
    
    return np.sum(weighted_vectors, axis=0) if weighted_vectors else np.zeros(word_vectors.vector_size)


In [18]:
def tfidf_weighted_sentence_embedding(text):
    words = text.lower().split()
    text_tfidf = vectorizer.transform([text])
    tfidf_scores = dict(zip(vocab, text_tfidf.toarray()[0]))
    
    weighted_vectors = []
    for word in words:
        if word in word_vectors and word in tfidf_scores:
            weighted_vectors.append(tfidf_scores[word] * word_vectors[word])
    
    # Return average of weighted word vectors -> sentence-level embedding
    if weighted_vectors:
        return np.mean(weighted_vectors, axis=0)
    else:
        return np.zeros(300)

def hybrid_sentence_embedding(text, alpha=0.5):
    """
    Hybrid sentence embedding: 
    - Part 1: TF-IDF weighted GloVe (sentence-level via averaging)
    - Part 2: spaCy's native sentence vector
    Combine using alpha.
    """
    # sentence-level representations
    lexical = tfidf_weighted_sentence_embedding(text)
    semantic = nlp(text).vector  # This is already sentence-level

    # Normalize before combination (try standardization next?)
    lexical_norm = lexical / (np.linalg.norm(lexical) + 1e-10)
    semantic_norm = semantic / (np.linalg.norm(semantic) + 1e-10)

    # merge
    return alpha * lexical_norm + (1 - alpha) * semantic_norm


In [22]:
# ===== DEMONSTRATION =====
sample_text = texts[0]

print("\nOriginal text:", sample_text)
print("Average embedding shape:", average_embedding(sample_text).shape)
print("TF-IDF weighted embedding shape:", tfidf_weighted_embedding(sample_text).shape)
print("Hybrid embedding shape:", hybrid_sentence_embedding(sample_text).shape)

print("tfidf embedding shape:", tfidf_weighted_sentence_embedding(sample_text).shape)


Original text: I hate chess, i know what chess is all about, it is about memorization and knowing patterns in positions
Average embedding shape: (300,)
TF-IDF weighted embedding shape: (300,)
Hybrid embedding shape: (300,)
tfidf embedding shape: (300,)


In [20]:
# similarity
emb1 = hybrid_sentence_embedding(texts[0])
emb2 = hybrid_sentence_embedding(texts[1])
similarity = np.dot(emb1, emb2) / (np.linalg.norm(emb1) * np.linalg.norm(emb2))
print(f"\nSimilarity between sentences: {similarity:.3f}")



Similarity between sentences: 0.839


In [23]:
# similarity
emb1 = tfidf_weighted_sentence_embedding(texts[0])
emb2 = tfidf_weighted_sentence_embedding(texts[1])
similarity = np.dot(emb1, emb2) / (np.linalg.norm(emb1) * np.linalg.norm(emb2))
print(f"\nSimilarity between sentences: {similarity:.3f}")


Similarity between sentences: 0.818


In [24]:
# similarity
emb1 = nlp(texts[0]).vector
emb2 = nlp(texts[1]).vector
similarity = np.dot(emb1, emb2) / (np.linalg.norm(emb1) * np.linalg.norm(emb2))
print(f"\nSimilarity between sentences: {similarity:.3f}")


Similarity between sentences: 0.903


In [17]:
emb1 = average_embedding(texts[0])
emb2 = average_embedding(texts[1])
similarity = np.dot(emb1, emb2) / (np.linalg.norm(emb1) * np.linalg.norm(emb2))
print(f"\nSimilarity between sentences: {similarity:.3f}")


Similarity between sentences: 0.794


In [16]:
emb1 = tfidf_weighted_embedding(texts[0])
emb2 = tfidf_weighted_embedding(texts[1])
similarity = np.dot(emb1, emb2) / (np.linalg.norm(emb1) * np.linalg.norm(emb2))
print(f"\nSimilarity between sentences: {similarity:.3f}")


Similarity between sentences: 0.818


In [13]:
print(average_embedding(sample_text))

[-2.03662992e-01  1.24967769e-01  4.10734043e-02 -1.14144877e-01
 -1.06489480e-01  4.72516157e-02 -2.40327809e-02 -8.84871855e-02
  1.30181191e-02 -1.65706670e+00  1.63147211e-01  3.80054712e-02
 -6.85284585e-02  2.54368246e-01  1.12130351e-01  4.52426523e-02
 -1.60791889e-01  6.06757477e-02  1.71245877e-02  8.12233761e-02
  2.77539194e-01  3.81483406e-01  4.07207608e-02  5.07635809e-02
 -1.43210143e-01 -1.05810454e-02 -6.25022082e-03 -1.79993048e-01
 -1.02376543e-01  3.56353447e-02 -5.38647640e-04  2.80988872e-01
 -2.47087985e-01 -1.24153897e-01 -8.27469409e-01  1.37498602e-01
 -1.46131758e-02 -2.25064415e-03 -2.13187426e-01 -2.15320028e-02
  1.71413481e-01 -1.54911339e-01 -5.13146864e-03 -6.64251670e-02
  8.23577940e-02  4.61672060e-02  2.55274445e-01  2.07291633e-01
 -5.24860919e-02  9.25543085e-02  2.46886864e-01 -1.13356352e-01
  1.48810267e-01 -1.14052944e-01 -3.03870946e-01  2.63287455e-01
  2.48160604e-02  1.27779633e-01  3.18156391e-01  1.33925170e-01
  2.44577542e-01  5.76964

In [14]:
print(tfidf_weighted_embedding(sample_text))

[-0.95437694  0.5469725   0.38252345 -0.58971184 -0.46116322  0.20439863
 -0.19794136 -0.29934993  0.04208573 -6.286654    0.57562333  0.2424912
 -0.23324376  1.0559671   0.650699    0.14808568 -0.86809623  0.33906624
  0.03711678  0.12558272  1.0161449   1.6018635  -0.09499978  0.6662142
 -0.32594532  0.039951   -0.0565544  -0.49994645 -0.74552065  0.43542412
 -0.0479335   1.1620617  -0.9082856  -0.36072925 -2.8273826   0.5901823
  0.08446629  0.10271606 -1.1460466  -0.26630208  0.8492764  -0.01820392
 -0.02461366 -0.08444783  0.3472967   0.11758827  0.79785407  0.7007614
 -0.5347372   0.20203656  1.1419541  -0.50747776  0.53723705 -0.24251041
 -1.0023772   0.7656495  -0.27241504  0.750928    1.2983564   0.35309517
  0.45672327  0.0371781   0.61641276  0.5684712  -0.20277476 -1.2120718
  0.15966469  0.41019666 -0.26049334  0.31161273 -0.4550906  -0.02316811
  0.04263891 -0.30855098 -0.4573682  -0.3360135   0.656826    0.6141375
 -0.9193255  -0.23756422 -0.35611594 -0.8634013   0.60654

In [21]:
hybrid_sentence_embedding(sample_text)

array([-1.26565307e-01,  3.92756015e-02, -2.14310251e-02, -3.74388546e-02,
       -2.20526662e-02,  2.23883316e-02,  1.60818603e-02, -4.05511335e-02,
        1.09057268e-02,  3.30435038e-02, -4.80044819e-03, -2.06855126e-04,
       -4.26090322e-03,  4.48861793e-02,  5.48415817e-03, -7.54403416e-03,
       -2.92741470e-02,  9.35750455e-02, -2.63516046e-02,  1.15981102e-02,
        4.22672033e-02,  5.04503660e-02,  3.84768075e-03, -8.67116638e-03,
       -2.12677494e-02, -3.30181909e-03, -1.88231282e-02, -3.34753841e-02,
       -2.17988640e-02, -7.42260925e-03, -1.25936437e-02,  6.04740940e-02,
       -5.43842092e-02, -3.65509875e-02, -9.25268754e-02,  4.95320633e-02,
        1.53624918e-04, -2.94920499e-03, -6.73998594e-02,  6.98742457e-03,
        2.55508516e-02, -1.48130609e-02, -1.60997361e-02,  9.60513111e-03,
        3.01918332e-02,  1.27040930e-02,  3.69562022e-02,  4.46079709e-02,
       -4.00808342e-02,  2.40126671e-03,  1.44084971e-02,  1.86166205e-02,
        3.68295982e-02, -